In [1]:
#r "nuget:Microsoft.ML,1.3.1"
#r "nuget:Microsoft.ML.Recommender, 0.15.1"
#r "nuget:XPlot.Plotly;"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML, 1.3.1 Microsoft.ML.Recommender, 0.15.1

In [2]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using XPlot.Plotly;

In [3]:
public class MovieRating
{
    [LoadColumn(0)]
    public float userId;
    [LoadColumn(1)]
    public float movieId;
    [LoadColumn(2)]
    public float Label;
}


(4,18): warning CS0649: Campo "MovieRating.userId" nunca é atribuído e sempre terá seu valor padrão 0

(8,18): warning CS0649: Campo "MovieRating.Label" nunca é atribuído e sempre terá seu valor padrão 0

(6,18): warning CS0649: Campo "MovieRating.movieId" nunca é atribuído e sempre terá seu valor padrão 0



In [4]:
public class MovieRatingPrediction
{
    public float Label;
    public float Score;
}


(3,18): warning CS0649: Campo "MovieRatingPrediction.Label" nunca é atribuído e sempre terá seu valor padrão 0

(4,18): warning CS0649: Campo "MovieRatingPrediction.Score" nunca é atribuído e sempre terá seu valor padrão 0



In [5]:
// Conceitos principais em ML.NET:  Data, Transformadores e estimadores. 

//Para treinar os dados, a máquina exite que os dados estejam em determinado formado. 
//Os Transformers são usados para transformar os dados de tabela em um formato compatível. 

//DATA -----> TRANFORMER ------> TRANSFORMED DATA

//Para criar esses Transformer você utilza os Estimators.
//Eles usam dados e retornam Transformers. 

//DATA -----> ESTIMATOR ----> TRANFORMER

//O algoritmo de treinamento de recomendação (por exemplo) que vai ser utilizado para treinar o modelo é um exemplo
//de Estimator.

In [6]:
// Carrega os dados
public static (IDataView training, IDataView test) LoadData(MLContext mlContext)
{
    // Carrega os dados de treino e teste usando os paths
    var trainingDataPath = Path.Combine(Environment.CurrentDirectory, "Data", "recommendation-ratings-train.csv");
    var testDataPath = Path.Combine(Environment.CurrentDirectory, "Data", "recommendation-ratings-test.csv");

    IDataView trainingDataView = mlContext.Data.LoadFromTextFile<MovieRating>(trainingDataPath, hasHeader: true, separatorChar: ',');
    IDataView testDataView = mlContext.Data.LoadFromTextFile<MovieRating>(testDataPath, hasHeader: true, separatorChar: ',');

    Console.WriteLine(trainingDataView.GetRowCount().ToString());
    Console.WriteLine(testDataView.GetRowCount().ToString());

    return (trainingDataView, testDataView);
}

In [7]:
// Cria e treina o modelo
public static ITransformer BuildAndTrainModel(MLContext mlContext, IDataView trainingDataView)
{
    // Adiciona os dados de Transformes
    IEstimator<ITransformer> estimator = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "userIdEncoded", inputColumnName: "userId")
        .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "movieIdEncoded", inputColumnName: "movieId"));

    var options = new MatrixFactorizationTrainer.Options
    {
        MatrixColumnIndexColumnName = "userIdEncoded",
        MatrixRowIndexColumnName = "movieIdEncoded", 
        LabelColumnName = "Label",
        NumberOfIterations = 20,
        ApproximationRank = 100
    };

    var trainerEstimator = estimator.Append(mlContext.Recommendation().Trainers.MatrixFactorization(options));

    Console.WriteLine("=============== Treinando o modelo ===============");
    ITransformer model = trainerEstimator.Fit(trainingDataView);

    return model;

}

In [12]:
// Avaliação do modelo (testes)
public static void EvaluateModel(MLContext mlContext, IDataView testDataView, ITransformer model)
{
    // Evaluate model on test data & print evaluation metrics
    Console.WriteLine("=============== Avaliando o modelo ===============");
    var prediction = model.Transform(testDataView);

    var metrics = mlContext.Regression.Evaluate(prediction, labelColumnName: "Label", scoreColumnName: "Score");

    // A root of mean squared error (RMS ou RMSE - Root Mean Squared Error) é usada para medir as diferenças entre os valores previstos do modelo e os valores observados do conjunto de dados de teste. Tecnicamente, ela é a raiz quadrada da média dos quadrados dos erros. Quanto menor, melhor o modelo.
    // R Squared (Ajute ao modelo) indica o quanto os dados se ajustam a um modelo. Varia de 0 a 1. Um valor de 0 significa que os dados são aleatórios ou não podem ser ajustados ao modelo. Um valor de 1 significa que o modelo corresponde exatamente aos dados. Você deseja que a pontuação R Squared esteja o mais próximo possível de 1.    
    
    Console.WriteLine("RMSE (Quadrado do erros - Quanto menor melhor) : " + metrics.RootMeanSquaredError.ToString());
    Console.WriteLine("RSquared (Ajuste ao modelo - Mais próx. de 1 melhor): " + metrics.RSquared.ToString());
}

In [13]:
public static void SaveModel(MLContext mlContext, DataViewSchema trainingDataViewSchema, ITransformer model)
{
    var modelPath = Path.Combine(Environment.CurrentDirectory, "Data", "MovieRecommenderModel.zip");

    Console.WriteLine("=============== Salvando arquivo do modelo ===============");
    mlContext.Model.Save(model, trainingDataViewSchema, modelPath);
}

In [14]:
public static void UseModelForSinglePrediction(MLContext mlContext, ITransformer model)
{
    Console.WriteLine("=============== Realizando predição ===============");
    var predictionEngine = mlContext.Model.CreatePredictionEngine<MovieRating, MovieRatingPrediction>(model);

    // Cria teste de entrada e faz uma simples predição
    // Create test input & make single prediction
    var testInput = new MovieRating { userId = 6, movieId = 10 };

    var movieRatingPrediction = predictionEngine.Predict(testInput);

    if (Math.Round(movieRatingPrediction.Score, 1) > 3.5)
    {
        Console.WriteLine("Filme " + testInput.movieId + " é recomendado para o usuário " + testInput.userId);
    }
    else
    {
        Console.WriteLine("Filme " + testInput.movieId + " não é recomendado para o usuário " + testInput.userId);
    }
}

In [16]:
// Programa principal

MLContext mlContext = new MLContext();

(IDataView trainingDataView, IDataView testDataView) = LoadData(mlContext);

ITransformer model = BuildAndTrainModel(mlContext, trainingDataView);

EvaluateModel(mlContext, testDataView, model);

UseModelForSinglePrediction(mlContext, model);

SaveModel(mlContext, trainingDataView.Schema, model);



=============== Treinando o modelo ===============
=============== Avaliando o modelo ===============
RMSE (Quadrado do erros - Quanto menor melhor) : 0,9740768910727712
RSquared (Ajuste ao modelo - Mais próx. de 1 melhor): 0,43592742042215304
=============== Realizando predição ===============
Filme 10 é recomendado para o usuário 6
=============== Salvando arquivo do modelo ===============
